### Weblink to use for dataset: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
# Webscrape link into pandas dataframe

In [2]:
#webscrape the link into pandas df
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from pandas.io.json import json_normalize

import numpy as np
from geopy.geocoders import Nominatim
import json
from bs4 import BeautifulSoup
import requests
print('Imported')

Imported


# Use BeautifulSoup to Transform Data

In [3]:
#Use BeautifulSoup to transform the data
link = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

#make beautifulsoup object
soup = BeautifulSoup(link, 'html.parser')

#create lists to store the data
postCodeList =[]
boroughList = []
neighborhoodList = []


# Add To the Lists

In [4]:
#add to the lists
for row in soup.find('table').find_all('tr'):
    cell = row.find_all('td')
    if(len(cell) > 0):
        postCodeList.append(cell[0].text)
        boroughList.append(cell[1].text)
        neighborhoodList.append(cell[2].text.rstrip('\n'))

# Read the Dataframe 

In [5]:
#Read the dataframe
canada_df = pd.DataFrame({'Postal Code': postCodeList,
                          'Borough': boroughList,
                          'Neighborhood': neighborhoodList})
canada_df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


# Drop Cells Labeled 'Not Assigned'

In [6]:
#Drop cells with 'Not assigned'
canada_df_drop = canada_df[canada_df.Borough != 'Not assigned'].reset_index(drop=True)
canada_df_drop.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


# Group Neighborhoods as Required

In [7]:
#using lambda to aggregate the neighborhoods; first time incorporating lambda
canada_df_group = canada_df_drop.groupby(['Postal Code', 'Borough'], as_index=False).agg(lambda x: ", ".join(x))
   
canada_df_group.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


# Assign the 'N/A' Neighborhoods to Borough

In [8]:
#assign not assigned neighborhood to respective borough. 
#iterate over the rows 

for ind, row in canada_df_group.iterrows():
    if row['Neighborhood'] == 'Not assigned':
        row['Neighborhood'] == row['Borough']
        
canada_df_group.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


# Print the Number of Rows

In [9]:
#print the number of rows
canada_df_group.shape

(103, 3)

# Download coordinate data

In [11]:
coord = pd.read_csv("https://cocl.us/Geospatial_data")
coord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


# Merge the postal data and coordinate data

In [12]:
canada_df_group2 = canada_df_group.merge(coord, on="Postal Code", how="left")
canada_df_group2.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
